In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
athletes_df = pd.read_csv('../hw1-athletes.csv')
athletes_df.head()

,name,sport,age,height,weight,country,sex,gold,silver,bronze,total
0,Lamusi A,Judo,23,170,60,People's Republic of China,M,0,0,0,0
1,A G Kruger,Athletics,33,193,125,United States of America,M,0,0,0,0
2,Jamale Aarrass,Athletics,30,187,76,France,M,0,0,0,0
3,Abdelhak Aatakni,Boxing,24,NaN,NaN,Morocco,M,0,0,0,0
4,Maria Abakumova,Athletics,26,178,85,Russian Federation,F,0,0,0,0


In [3]:
print(len(athletes_df))
athletes = athletes_df.dropna()
print(len(athletes))

10383
9038


In [4]:
feature_cols = ['age', 'height', 'weight']
target_col = ['sport']
athletes[feature_cols][0:20]

,age,height,weight
0,23,170,60
1,33,193,125
2,30,187,76
4,26,178,85
5,27,182,80
6,30,182,73
7,23,187,75
8,27,190,80
12,28,173,60
13,28,171,64


In [5]:
X = athletes[feature_cols].values
print(X)
y = athletes[target_col].values.ravel()
print(y)
print(X.shape, y.shape)

[[  23.  170.   60.]
 [  33.  193.  125.]
 [  30.  187.   76.]
 ..., 
 [  27.  180.   87.]
 [  25.  173.   62.]
 [  24.  221.  110.]]
['Judo' 'Athletics' 'Athletics' ..., 'Handball' 'Athletics' 'Basketball']
(9038, 3) (9038,)


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y.ravel())

LabelEncoder()

In [7]:
le.classes_

array(['Archery', 'Athletics', 'Badminton', 'Basketball',
       'Beach Volleyball', 'Canoe Slalom', 'Canoe Sprint', 'Cycling',
       'Cycling - BMX', 'Cycling - Mountain Bike', 'Cycling - Road',
       'Cycling - Track', 'Diving', 'Equestrian', 'Fencing', 'Football',
       'Handball', 'Hockey', 'Judo', 'Modern Pentathlon', 'Rowing',
       'Sailing', 'Shooting', 'Swimming', 'Table Tennis', 'Tennis',
       'Triathlon', 'Volleyball', 'Water Polo', 'Weightlifting',
       'Wrestling'], dtype=object)

In [8]:
print(y)
target = le.transform(y).ravel()
print(target)

['Judo' 'Athletics' 'Athletics' ..., 'Handball' 'Athletics' 'Basketball']
[18  1  1 ..., 16  1  3]


In [9]:
xn = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])

nbrs = NearestNeighbors(n_neighbors=3, metric='euclidean').fit(xn)
nbrs.kneighbors([0,0])

/Users/soleiladler/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([[ 1.41421356,  1.41421356,  2.23606798]]), array([[0, 3, 1]]))

In [10]:
#result is three distances, and three indices of points
#[-1, -1], [-2, -1],and [1,1] are the closest points to (0,0)

In [11]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [12]:
X_test = np.array([[23., 170., 60.], [33., 193., 125.]])
knn.predict(X_test)

array(['Wrestling', 'Athletics'], dtype=object)

In [13]:
from sklearn import metrics

y_pred = knn.predict(X)
metrics.accuracy_score(y, y_pred)

#We should be getting 100% because we are predicting the points that we have!
#BUT! We get 80% because there are some athletes that have the same sizes
#and age in different sports!

0.80792210666076569

In [14]:
y_pred

array(['Wrestling', 'Athletics', 'Athletics', ..., 'Water Polo',
       'Athletics', 'Basketball'], dtype=object)

In [15]:
y

array(['Judo', 'Athletics', 'Athletics', ..., 'Handball', 'Athletics',
       'Basketball'], dtype=object)

In [18]:
#They are different because certain types of athletes have the same stats.
#We just did it on points that we know the answer to. Now, we will take out the points we know
#by using Cross Validation

In [19]:
from sklearn.cross_validation import LeaveOneOut, cross_val_score

loo = LeaveOneOut(len(y))
c_score = cross_val_score(knn, X, y, cv=loo)

In [20]:
c_score

array([ 0.,  1.,  0., ...,  0.,  0.,  1.])

In [21]:
(sum(c_score))/len(c_score)

0.17946448329276388

In [22]:
from sklearn.cross_validation import KFold

FEATURE_COLS = ['age', 'height', 'weight']
TARGET_COL = ['sport']

Xtest = athletes[FEATURE_COLS].values
ytest = athletes[TARGET_COL].values.ravel()

# Create the model
knntest = KNeighborsClassifier(n_neighbors=1, metric='euclidean');
knntest.fit(Xtest, ytest);

# Cross-validation
loo = KFold(n=len(Xtest), n_folds=10);
cross_val_score(knntest, Xtest, ytest, cv=loo)

array([ 0.1880531 ,  0.19026549,  0.18141593,  0.15044248,  0.16814159,
        0.16150442,  0.19358407,  0.17809735,  0.17497231,  0.1683278 ])